In [1]:

import sys
import os
sys.path.append('/home/ubuntu/models_implem')
# os.environ["TORCH_LOGS"] = "+dynamic"
# os.environ["TORCHDYNAMO_VERBOSE"] = "1"

from transformers import (
    AutoImageProcessor,
    RTDetrForObjectDetection,
    RTDetrV2ForObjectDetection,
    DetrForObjectDetection,
    DetrImageProcessorFast,
    AutoModelForObjectDetection,
)
import torch
from PIL import Image
import requests
from e2e_implem.debug_utils import debug_dynamic_shapes_pipeline
from e2e_implem.e2e_utils import E2EModel

# torch._dynamo.config.capture_dynamic_output_shape_ops = True

# model_id = "meta-llama/Llama-4-Maverick-17B-128E-Instruct"
# model_id = "PekingU/rtdetr_v2_r101vd"
# model_id = "PekingU/rtdetr_r50vd_coco_o365"
model_id = "ustc-community/dfine-large-obj365"
# model_id = "hustvl/yolos-tiny"
# model_id = "facebook/detr-resnet-50"
image = Image.open(requests.get("http://images.cocodataset.org/val2017/000000039769.jpg", stream=True).raw)
image_big = image.resize((1024, 1024))


🚨 `intermediate_return` is part of BaseImageProcessorFast.preprocess's signature, but not documented. Make sure to add it to the docstring of the function in /home/ubuntu/models_implem/transformers/src/transformers/image_processing_utils_fast.py.


In [2]:
e2e_rtdetr = E2EModel(model_id, AutoModelForObjectDetection, AutoImageProcessor, "post_process_object_detection").to(
    "cpu"
)
images = e2e_rtdetr.get_tensors_inputs(image, device="cpu")
target_sizes = torch.tensor([image.size[::-1]])
inputs = {"images": images, "post_process_kwargs": {"target_sizes": target_sizes}}

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
debug_dynamic_shapes_pipeline(e2e_rtdetr,inputs, model_name=model_id)


DEBUGGING DYNAMIC SHAPES PIPELINE FOR: ustc-community/dfine-large-obj365

1️⃣ TESTING torch.export (MOST STRICT)
--------------------------------------------------
   🔄 Attempting torch.export with dynamic_shapes...
   📋 Dynamic shapes: {'images': {2: <class 'e2e_implem.debug_utils.height'>, 3: <class 'e2e_implem.debug_utils.width'>}, 'post_process_kwargs': {'target_sizes': None}}


   ✅ torch.export SUCCESS in 5.30s
   📊 Exported graph has 3971 nodes
   🧪 Testing exported program with different sizes...


/opt/conda/envs/hf_311_121/lib/python3.11/site-packages/torch/export/_unlift.py:75: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  getattr_node = gm.graph.get_attr(lifted_node)
/opt/conda/envs/hf_311_121/lib/python3.11/site-packages/torch/fx/graph.py:1801: UserWarning: Node model_model_lifted_tensor_0 target model.model.lifted_tensor_0 lifted_tensor_0 of model.model does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/opt/conda/envs/hf_311_121/lib/python3.11/site-packages/torch/fx/graph.py:1801: UserWarning: Node model_model_lifted_tensor_1 target model.model.lifted_tensor_1 lifted_tensor_1 of model.model does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_a

   📈 Size test results: ✅ (224, 224), ✅ (480, 640), ✅ (800, 600)

3️⃣ TESTING ONNX EXPORT (FINAL CONVERSION)
--------------------------------------------------
   🔄 Attempting ONNX export...
     🔄 Trying Dynamo with dynamic_shapes...
[torch.onnx] Obtain model graph for `E2EModel([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `E2EModel([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


/opt/conda/envs/hf_311_121/lib/python3.11/site-packages/torch/export/_unlift.py:75: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  getattr_node = gm.graph.get_attr(lifted_node)
/opt/conda/envs/hf_311_121/lib/python3.11/site-packages/torch/fx/graph.py:1801: UserWarning: Node model_model_lifted_tensor_0 target model.model.lifted_tensor_0 lifted_tensor_0 of model.model does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/opt/conda/envs/hf_311_121/lib/python3.11/site-packages/torch/fx/graph.py:1801: UserWarning: Node model_model_lifted_tensor_1 target model.model.lifted_tensor_1 lifted_tensor_1 of model.model does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_a

...

W0915 20:17:46.968000 698851 site-packages/torch/onnx/_internal/exporter/_core.py:1134] Tensor 'model.model.lifted_tensor_0' is not one of the initializers
W0915 20:17:46.968000 698851 site-packages/torch/onnx/_internal/exporter/_core.py:1134] Tensor 'model.model.lifted_tensor_1' is not one of the initializers
W0915 20:17:46.969000 698851 site-packages/torch/onnx/_internal/exporter/_core.py:1134] Tensor 'model.model.lifted_tensor_2' is not one of the initializers
W0915 20:17:46.970000 698851 site-packages/torch/onnx/_internal/exporter/_core.py:1134] Tensor 'model.model.lifted_tensor_3' is not one of the initializers
W0915 20:17:46.970000 698851 site-packages/torch/onnx/_internal/exporter/_core.py:1134] Tensor 'model.model.lifted_tensor_4' is not one of the initializers
W0915 20:17:46.971000 698851 site-packages/torch/onnx/_internal/exporter/_core.py:1134] Tensor 'model.model.lifted_tensor_5' is not one of the initializers


[torch.onnx] Translate the graph into ONNX... ✅
     ✅ Dynamo with dynamic_shapes SUCCESS
     🔄 Optimizing ONNX model...
Applied 47 of general pattern rewrite rules.
     ✅ ONNX model optimized
     📋 ONNX model saved to ustc-community_dfine-large-obj365_dynamo_with_dynamic_shapes.onnx
     🧪 Verifying ONNX model with different sizes...


2025-09-15 20:17:52.290252879 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'linear_28' source:{1200,1} target:{1200}. Falling back to lenient merge.
2025-09-15 20:17:52.290933509 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'linear_43' source:{1200,1} target:{1200}. Falling back to lenient merge.
2025-09-15 20:17:52.291499486 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'linear_58' source:{1200,1} target:{1200}. Falling back to lenient merge.
2025-09-15 20:17:52.292058922 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'linear_73' source:{1200,1} target:{1200}. Falling back to lenient merge.
2025-09-15 20:17:52.292627589 [W:onnxruntime:, graph.cc:117 MergeShapeInfo] Error merging shape info for output. 'linear_88' source:{1200,1} target:{1200}. Falling back to lenient merge.
2025-09-15 20:17:52.293337440 [W:onnxruntime:, graph.cc:117 Merge

     📈 ONNX Runtime test results: ✅ (224, 224), ✅ (480, 640), ✅ (800, 600)

4️⃣ ANALYSIS & RECOMMENDATIONS
--------------------------------------------------

📊 RESULTS SUMMARY:
   ✅ torch_export: SUCCESS
   ✅ onnx_Dynamo with dynamic_shapes: SUCCESS
   ✅ onnx_Dynamo with dynamic_shapes_runtime: SUCCESS

💡 DIAGNOSIS:
   ✅ Model architecture supports dynamic shapes
   ⚠️  Model supports dynamic shapes but ONNX conversion fails
   💡 Try: different opset versions, disable constant folding, or use torch.export → onnx conversion



{'torch_export': 'SUCCESS',
 'onnx_Dynamo with dynamic_shapes': 'SUCCESS',
 'onnx_Dynamo with dynamic_shapes_runtime': 'SUCCESS'}